In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [5]:
# Load your dataset
df = pd.read_csv("match_history.csv")  #Reminder put file in same file

In [6]:
# Input and output columns
input_cols = ['CharacterName', 'Weapon']
output_cols = ['Item_Weapon', 'Item_Head', 'Item_Chest', 'Item_Arm', 'Item_Leg']

In [7]:
# Encode inputs
input_encoders = {col: LabelEncoder().fit(df[col]) for col in input_cols}
X = pd.DataFrame({col: input_encoders[col].transform(df[col]) for col in input_cols})

In [8]:
# Encode outputs
output_encoders = {col: LabelEncoder().fit(df[col]) for col in output_cols}
y = pd.DataFrame({col: output_encoders[col].transform(df[col]) for col in output_cols})

In [9]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Train a multi-output random forest classifier
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
model.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [11]:
# Evaluate performance
y_pred = model.predict(X_test)
for i, col in enumerate(output_cols):
    print(f"--- {col} ---")
    print(classification_report(y_test[col], y_pred[:, i]))

--- Item_Weapon ---
              precision    recall  f1-score   support

           1       0.50      1.00      0.67         4
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.50      1.00      0.67         2
           6       0.50      1.00      0.67         2
          14       1.00      1.00      1.00         1
          19       0.00      0.00      0.00         0
          21       0.00      0.00      0.00         0
          23       0.00      0.00      0.00         1
          26       0.33      1.00      0.50         1
          28       0.20      1.00      0.33         1
          31       0.00      0.00      0.00         0
          32       0.50      1.00      0.67         1
          34       1.00      1.00      1.00         1
          36       0.00      0.00      0.00         1
          38       0.00      0.00      0.00         0
       

c:\Users\Zai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Zai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Zai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{met

Accuracy check

In [12]:
# Accuracy per item slot
for i, col in enumerate(output_cols):
    acc = accuracy_score(y_test[col], y_pred[:, i])
    print(f"{col} Accuracy: {acc:.2%}")

# Exact match accuracy (all 5 items must match)
exact_match_acc = (y_test.values == y_pred).all(axis=1).mean()
print(f"\nExact Match Accuracy (full build): {exact_match_acc:.2%}")

Item_Weapon Accuracy: 48.61%
Item_Head Accuracy: 38.89%
Item_Chest Accuracy: 44.44%
Item_Arm Accuracy: 34.72%
Item_Leg Accuracy: 25.00%

Exact Match Accuracy (full build): 8.33%


EXAMPLE

In [20]:
example_character = "Aya"
example_weapon = "Pistol"

if example_character not in input_encoders['CharacterName'].classes_:
    raise ValueError(f"Character '{example_character}' not found in training data.")

if example_weapon not in input_encoders['Weapon'].classes_:
    raise ValueError(f"Weapon '{example_weapon}' not found in training data.")

char_encoded = input_encoders['CharacterName'].transform([example_character])[0]
weap_encoded = input_encoders['Weapon'].transform([example_weapon])[0]

prediction = model.predict([[char_encoded, weap_encoded]])[0]

decoded_items = {
    item: output_encoders[item].inverse_transform([pred])[0]
    for item, pred in zip(output_cols, prediction)
}

print("Recommended Build:")
print(decoded_items)


Recommended Build:
{'Item_Weapon': 'Altair', 'Item_Head': 'Persona', 'Item_Chest': 'Bikini', 'Item_Arm': 'Bracelet_of_Skadi', 'Item_Leg': 'Delta_Red'}


c:\Users\Zai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Zai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Zai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Zai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Zai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature n

In [14]:
print("Valid Characters:", input_encoders['CharacterName'].classes_)
print("Valid Weapons:", input_encoders['Weapon'].classes_)


Valid Characters: ['Abigail' 'Adela' 'Adriana' 'Alex' 'Alonso' 'Arda' 'Aya' 'Bernice'
 'Bianca' 'Cathy' 'Celine' 'Charlotte' 'Chiara' 'Chloe' 'Daniel' 'Darko'
 'Debi_&_Marlene' 'Echion' 'Elena' 'Eleven' 'Emma' 'Estelle' 'Eva' 'Fiora'
 'Garnet' 'Hart' 'Haze' 'Hisui' 'Hyejin' 'Hyunwoo' 'Irem' 'Isaac' 'Isol'
 'Istvan' 'Jackie' 'Jenny' 'Johann' 'Justyna' 'Katja' 'Kenneth' 'Laura'
 'Leni' 'Lenore' 'Lenox' 'Leon' 'Li_Dailin' 'Luke' 'Ly_Anh' 'Magnus' 'Mai'
 'Markus' 'Nadine' 'Nathapon' 'Nicky' 'Piolo' 'Priya' 'Rio' 'Rozzi'
 'Shoichi' 'Silvia' 'Sissela' 'Sua' 'Tazia' 'Theodore' 'Tia' 'Tsubame'
 'Vanya' 'William' 'Xiukai' 'Yuki' 'Yumin' 'Zahir' nan]
Valid Weapons: ['Arcana' 'Assault_Rifle' 'Axe' 'Bat' 'Bow' 'Camera' 'CrossBow' 'Dagger'
 'Dual_Swords' 'Glove' 'Guitar' 'Hammer' 'Nunchaku' 'Pistol' 'Rapier'
 'Shuriken' 'Sniper_Rifle' 'Spear' 'Throw' 'Tonfa' 'Two-handed_Sword'
 'VF_Prosthetics' 'Whip' nan]
